In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

df = yf.download('AAPL', start='2010-01-01', end='2023-12-31')
df.reset_index(inplace=True)
df = df.drop('Date', axis=1)
print("Data loaded:", df.shape)

C:\Users\bgay9\AppData\Local\Temp\ipykernel_6460\669374294.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download('AAPL', start='2010-01-01', end='2023-12-31')
[*********************100%***********************]  1 of 1 completed

Data loaded: (3522, 5)



C:\Users\bgay9\AppData\Local\Temp\ipykernel_6460\669374294.py:10: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df = df.drop('Date', axis=1)


In [3]:
training_size = int(len(df) * 0.8)
data_training = pd.DataFrame(df['Close'][0:training_size])

scaler = MinMaxScaler(feature_range=(0,1))
data_training_scaled = scaler.fit_transform(data_training)

x_train = []
y_train = []

for i in range(100, len(data_training_scaled)):
    x_train.append(data_training_scaled[i-100:i, 0])
    y_train.append(data_training_scaled[i, 0])

x_train = np.array(x_train)
y_train = np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

print("Training data prepared:", x_train.shape, y_train.shape)

Training data prepared: (2717, 100, 1) (2717,)


In [4]:
# Build model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=60, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(units=80, return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(units=120))
model.add(Dropout(0.5))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')
print("Model built successfully")

c:\Users\bgay9\OneDrive\Desktop\stock\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model built successfully


In [5]:
# Train and save
model.fit(x_train, y_train, epochs=10, batch_size=32, verbose=1)  # Reduced epochs for testing
model.save('keras_model.h5')
print("Model saved successfully!")


Epoch 1/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 11s 84ms/step - loss: 0.0185
Epoch 2/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 7s 84ms/step - loss: 0.0022
Epoch 3/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 7s 83ms/step - loss: 0.0028
Epoch 4/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 7s 83ms/step - loss: 0.0027
Epoch 5/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 7s 83ms/step - loss: 0.0018
Epoch 6/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 7s 83ms/step - loss: 0.0016
Epoch 7/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 7s 83ms/step - loss: 0.0020
Epoch 8/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 7s 84ms/step - loss: 0.0014
Epoch 9/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 7s 84ms/step - loss: 0.0021
Epoch 10/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 7s 84ms/step - loss: 0.0013


Model saved successfully!


In [6]:
# Test if model loads
from keras.models import load_model
try:
    test_model = load_model('keras_model.h5')
    print("Model loads correctly!")
except Exception as e:
    print("Error loading model:", e)

Model loads correctly!
